In [65]:
import pandas as pd
import numpy as np
import re
from transformers import AutoModel, AutoTokenizer
import torch

In [4]:
def CleanText(text):
    text = re.sub(r'''[\[|\]]''', "", text).split()
    text = np.array(text, dtype="float64")
    return text

# Normal dataset
df1 = pd.read_csv("all_data_with_identitiesEmbedded.csv")
df1['comment_text'] = df1['comment_text'].apply(lambda text: CleanText(text))
df1['toxicity'] = df1['toxicity'].apply(lambda text: round(text))

# Normal dataset
#df2 = pd.read_csv("all_data_with_identities.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'all_data_with_identitiesEmbedded.csv'

In [ ]:
df = pd.read_csv("../CSVFiles/all_data_with_identities.csv")

In [68]:
# Normal dataset
df = pd.read_csv("../CSVFiles/all_data_with_identities.csv")
df['toxicity'] = df['toxicity'].apply(lambda text: np.round(text >= 0.5))

#df['male'] = df['male'].apply(lambda x: np.round(x >= 0.5))
#df['female'] = df['female'].apply(lambda x: np.round(x >= 0.5))
#df['LGBTQ'] = df['LGBTQ'].apply(lambda x: np.round(x >= 0.5))
#df['christian'] = df['christian'].apply(lambda x: np.round(x >= 0.5))
#df['muslim'] = df['muslim'].apply(lambda x: np.round(x >= 0.5))
#df['black'] = df['black'].apply(lambda x: np.round(x >= 0.5))
#df['white'] = df['white'].apply(lambda x: np.round(x >= 0.5))
#df['other_religion'] = df['other_religion'].apply(lambda x: np.round(x >= 0.5))

#df = df.loc[:, ["comment_text", "split", "toxicity", "male", "female", "LGBTQ", "christian", "muslim", "other_religion", "black", "white"]]



In [69]:
df

,Unnamed: 0,id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,...,asian_latino_etc,disability_any,identity_any,num_identities,more_than_one_identity,na_gender,na_orientation,na_religion,na_race,na_disability
0,0,627762,OH yes - Were those evil Christian Missionarie...,test,2016-11-26 15:56:03.862109+00,13,627198.0,152737,approved,0,...,0,0,1,1.0,False,1,1,0,1,1
1,1,5892815,Why is this black racist crap still on the G&M...,val,2017-09-03 23:20:08.226613+00,54,NaN,373428,rejected,0,...,0,0,1,2.0,True,1,1,1,0,1
2,2,416437,even up here.......BLACKS!,train,2016-08-04 16:48:07.175252+00,21,NaN,143025,rejected,0,...,0,0,1,1.0,False,1,1,1,0,1
3,3,5137126,Blame men. There's always an excuse to blame ...,train,2017-04-15 19:00:45.032674+00,54,5136907.0,327125,rejected,0,...,0,0,1,2.0,True,0,1,1,1,1
4,4,855753,And the woman exposing herself saying grab thi...,val,2017-01-18 01:50:57.478867+00,13,849081.0,162008,rejected,0,...,0,0,1,1.0,False,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447995,447995,1018736,Another man shamming article. If white men did...,train,2017-02-20 07:20:49.964620+00,54,NaN,169202,approved,0,...,0,0,1,2.0,True,0,1,1,0,1
447996,447996,340016,"""no matter what is put in front of you regardi...",train,2016-06-06 06:43:04.780968+00,21,339965.0,137961,approved,0,...,0,0,1,2.0,True,0,1,1,1,1
447997,447997,919629,The Democrat party aided and abetted by it's M...,test,2017-01-30 02:44:29.168863+00,54,NaN,164845,rejected,0,...,0,0,0,0.0,False,1,1,1,1,1
447998,447998,5165492,I just don't find her a very good representati...,train,2017-04-22 18:42:02.442987+00,54,NaN,328877,approved,1,...,0,0,1,1.0,False,0,1,1,1,1


In [82]:
idxLst = []
splitLst = []

idxLst.extend(df.query('other_religion>=0.5 and toxicity == 1').index[:30])
for i in range(10):
    splitLst.append('train')
    splitLst.append('test')
    splitLst.append('val')
    

idxLst.extend(df.query('other_religion>=0.5 and toxicity == 0').index[:30])
for i in range(10):
    splitLst.append('train')
    splitLst.append('test')
    splitLst.append('val')
    
    
toxicity = [1, 0]

for t in toxicity:
    count = 0
    iterater = 0
    currentDF = df.query('toxicity=='+str(t))
    currentDFIndexes = currentDF.index
    while count < 25000:
        if currentDFIndexes[iterater] not in idxLst:
            idxLst.append(currentDFIndexes[iterater])
            count+=1
            if count <= 22000:
                splitLst.append('train')
            elif count > 22000 and count <= 23500:
                splitLst.append('test')
            else:
                splitLst.append('val')
        iterater +=1
    


In [74]:
idxLst = []
splitLst = []

idxLst.extend(df.query('other_religion>=0.5 and toxicity == 1').index[:30])
for i in range(10):
    splitLst.append('train')
    splitLst.append('test')
    splitLst.append('val')
    

idxLst.extend(df.query('other_religion>=0.5 and toxicity == 0').index[:30])
for i in range(10):
    splitLst.append('train')
    splitLst.append('test')
    splitLst.append('val')
    
    
print(len(idxLst))
print(len(splitLst))


#categories = ['male', 'christian', 'LGBTQ', 'muslim', 'female', 'black', 'white']
toxicity = [1, 0]

for t in toxicity:
    for cat in categories:
        
        count = 0
        iterater = 0
        
        currentDF = df.query(cat+'>=0.5 and toxicity=='+str(t))
        currentDFIndexes = currentDF.index
        while count < 4000:
            if currentDFIndexes[iterater] not in idxLst:
                idxLst.append(currentDFIndexes[iterater])
                count+=1
                if count <= 3000:
                    splitLst.append('train')
                elif count > 3000 and count <= 3500:
                    splitLst.append('test')
                else:
                    splitLst.append('val')
            iterater +=1
        print(iterater)

60
60
4002


IndexError: index 4090 is out of bounds for axis 0 with size 4090

In [83]:
print(len(idxLst))

50060


In [84]:
newDF = df.loc[idxLst]

for i, idx in enumerate(idxLst):
    newDF.at[idx, 'split'] = splitLst[i]

In [86]:
newDF.to_csv(r'C:\Users\frede\Desktop\small50000AllDataWithIdentities.csv', index=False, header=True)

In [85]:
tempDF = newDF.loc[:, ["comment_text", "split", "toxicity", "male", "female", "LGBTQ", "christian", "muslim", "other_religion", "black", "white"]]

print('Toxic samples: ', sum(newDF['toxicity']))
print('None-toxic samples: ', len(newDF['toxicity'])-sum(newDF['toxicity']))

print("\n")

print("Validation samples: ", np.sum(newDF['split'] == 'val'))
print("Test samples", np.sum(newDF['split'] == 'test'))

print("\n")

for col in tempDF.columns[3:]:
    print(col + ": " + str(np.sum(tempDF[col])))

Toxic samples:  25030.0
None-toxic samples:  25030.0


Validation samples:  3020
Test samples 3020


male: 5159.836288897452
female: 6026.481274256372
LGBTQ: 1945
christian: 2740.217357092696
muslim: 2646.730185007008
other_religion: 216.70208189093648
black: 2660.136958801574
white: 4044.2291943775836


In [87]:
import pandas as pd
import numpy as np
import re
from transformers import AutoModel, AutoTokenizer
import torch



newDF = newDF.loc[:, ["comment_text", "split", "toxicity", "male", "female", "LGBTQ", "christian", "muslim", "other_religion", "black", "white"]]

newDF['male'] = newDF['male'].apply(lambda x: np.round(x >= 0.5))
newDF['female'] = newDF['female'].apply(lambda x: np.round(x >= 0.5))
newDF['LGBTQ'] = newDF['LGBTQ'].apply(lambda x: np.round(x >= 0.5))
newDF['christian'] = newDF['christian'].apply(lambda x: np.round(x >= 0.5))
newDF['muslim'] = newDF['muslim'].apply(lambda x: np.round(x >= 0.5))
newDF['black'] = newDF['black'].apply(lambda x: np.round(x >= 0.5))
newDF['white'] = newDF['white'].apply(lambda x: np.round(x >= 0.5))
newDF['other_religion'] = newDF['other_religion'].apply(lambda x: np.round(x >= 0.5))


dfToEmbedding= newDF

bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)                     
bertweet.eval()
bertweet.to('cuda')

with torch.no_grad():
    for i, idx in enumerate(dfToEmbedding.index):
        bertweet.resize_token_embeddings(len(tokenizer))
        dfToEmbedding.at[idx, 'comment_text'] = str(bertweet(torch.tensor(tokenizer.encode
                                                                  (dfToEmbedding['comment_text'][idx], add_special_tokens=True,
                                                                   truncation=True)).to('cuda').unsqueeze(0))[1].detach().cpu().numpy()[0])

dfToEmbedding.to_csv(r'C:\Users\frede\Desktop\50000DomainDataBertweetEmbedded.csv', index=False, header=True)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
import pandas as pd
import numpy as np
import re
from transformers import AutoModel, AutoTokenizer
import torch

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

dfToEmbedding= pd.read_csv("../CSVFiles/smallDomainDataNotEmbedded.csv")


bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)                     
bertweet.eval()
bertweet.to('cuda')

with torch.no_grad():
    for i, idx in enumerate(dfToEmbedding.index):
        embeds = bertweet(torch.tensor(tokenizer.encode(dfToEmbedding['comment_text'][idx], add_special_tokens=True,
                                                                   truncation=True)).to('cuda').unsqueeze(0))[0].squeeze(0)
        
        if embeds.shape[0] < 200:
            embeds = torch.cat((embeds, torch.zeros((200-embeds.shape[0], 768)).to('cuda')))
            
        elif embeds.shape[0] > 200:
            embeds = embeds[0:200]
        
        dfToEmbedding.at[idx, 'comment_text'] = embeds.detach().cpu().numpy()



#dfToEmbedding.to_csv(r'C:\Users\frede\Desktop\smallDomainDataBertweetWordEmbeds.csv', index=False, header=True)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Updated Df


In [8]:
dfToEmbedding['comment_text']

0       [[-0.018730633, -0.044255268, 0.08673429, -0.0...
1       [[-0.012146702, 0.23959883, 0.14874044, -0.115...
2       [[-0.04316508, 0.2109584, 0.083563395, -0.0394...
3       [[-0.102337755, 0.21968453, 0.2083544, -0.1189...
4       [[-0.19849724, 0.0055802, 0.12685396, -0.27278...
                              ...                        
2855    [[-0.15536311, 0.11322548, 0.14425291, -0.1191...
2856    [[-0.112591244, 0.03947962, 0.2966754, -0.3437...
2857    [[-0.10124668, 0.15868574, 0.22361813, -0.1241...
2858    [[0.034709007, 0.13317995, 0.2726409, -0.31985...
2859    [[0.04770079, 0.1619968, 0.19023469, -0.242425...
Name: comment_text, Length: 2860, dtype: object

In [9]:
X_train = np.array(dfToEmbedding['comment_text'].values.tolist())
Y_train = np.array(dfToEmbedding['toxicity'].values.tolist())